### **DEPENDENCIES**

In [1]:
%load_ext autoreload

In [2]:
import sys
import time
sys.path.append("..")
%autoreload 2
import team_ninety_one as tno

from guidance import models

### **ENV & CONFIG**

In [3]:
openai_settings = tno.settings.OpenAISettings()

### **MODEL**

In [4]:
top_lm = models.OpenAI(
    model="gpt-4o-mini",
    # model="gpt-3.5-turbo-instruct",
    api_key=openai_settings.api_key,
    echo=True,
)

### **HyDE**

In [9]:
query = "What color shirt did I wear 2 days ago?"
start = time.time()
lm_hyde = top_lm + tno.programs.hyde_g(query, max_tokens=200)
print(lm_hyde["hyde"])
print(f"Time taken: {time.time() - start}")
start = time.time()
hyde_answer = tno.programs.hyde_openai_chat(query, max_tokens=200)
print(hyde_answer)
print(f"Time taken: {time.time() - start}")

NO
Time taken: 0.026063203811645508
NO
Time taken: 1.3794279098510742
